In [2]:
%pip install -e .. pandas scikit-learn scipy statsmodels
from pathlib import Path
import pandas as pd
pd.set_option("display.max_rows", 100)

Obtaining file:///C:/Users/Linda/Documents/00-linda/02-project/trees
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 8.9/8.9 MB 78.4 MB/s  0:00:00
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   ------------------- -------------------- 19.7/41.3 MB 95.1 MB/s eta 0:00:01
   -----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires pydantic>=2.9.2, which is not installed.
doclayout-yolo 0.0.2b1 requires pillow>=7.1.2, which is not installed.
doclayout-yolo 0.0.2b1 requires torch>=2.0.1, which is not installed.
doclayout-yolo 0.0.2b1 requires torchvision>=0.15.2, which is not installed.
doclayout-yolo 0.0.2b1 requires tqdm>=4.64.0, which is not installed.


In [1]:
import pandas as pd, sklearn, scipy, statsmodels
import trees_experiments
print("OK:", pd.__version__, sklearn.__version__)


OK: 2.2.3 1.7.2


In [2]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("../data/raw")
files = {
    "arrhythmia": DATA_DIR / "arrhythmia.csv",
    "BCP": DATA_DIR / "BCP.csv",
    "website_phishing": DATA_DIR / "website-phishing.csv",
}

# Load datasets into dict dfs
dfs = {}
for name, path in files.items():
    # try comma, fallback to semicolon if needed
    try:
        df = pd.read_csv(path)
    except Exception:
        df = pd.read_csv(path, sep=";")
    dfs[name] = df
    print(f"{name}: {df.shape} -> {path.name}")

arrhythmia: (452, 280) -> arrhythmia.csv
BCP: (683, 11) -> BCP.csv
website_phishing: (11055, 31) -> website-phishing.csv


In [4]:
# Review all 3 datasets
for name, df in dfs.items():
    print(f"\n{name}-info:")
    df.info()
    display(df.head(10).style.set_caption(f"{name} — HEAD (first 10 rows)"))


arrhythmia-info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Columns: 280 entries, age to class
dtypes: float64(120), int64(155), object(5)
memory usage: 988.9+ KB



BCP-info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Sample code number           683 non-null    int64
 1   Clump Thickness              683 non-null    int64
 2   Uniformity of Cell Size      683 non-null    int64
 3   Uniformity of Cell Shape     683 non-null    int64
 4   Marginal Adhesion            683 non-null    int64
 5   Single Epithelial Cell Size  683 non-null    int64
 6   Bare Nuclei                  683 non-null    int64
 7   Bland Chromatin              683 non-null    int64
 8   Normal Nucleoli              683 non-null    int64
 9   Mitoses                      683 non-null    int64
 10  Class                        683 non-null    int64
dtypes: int64(11)
memory usage: 58.8 KB


,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2



website_phishing-info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   having_IP_Address             11055 non-null  int64
 1    URL_Length                   11055 non-null  int64
 2    Shortining_Service           11055 non-null  int64
 3    having_At_Symbol             11055 non-null  int64
 4    double_slash_redirecting     11055 non-null  int64
 5    Prefix_Suffix                11055 non-null  int64
 6    having_Sub_Domain            11055 non-null  int64
 7    SSLfinal_State               11055 non-null  int64
 8    Domain_registeration_length  11055 non-null  int64
 9    Favicon                      11055 non-null  int64
 10   port                         11055 non-null  int64
 11   HTTPS_token                  11055 non-null  int64
 12   Request_URL                  11055 non-null  int64
 13   URL_of

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Class
0,-1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,1,-1,1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,1,-1,1,0,-1,-1,-1,-1,0,1,1,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,0,-1,1,1,0,1,1,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,-1,1,-1,1,-1,-1,0,-1,1,1,1,1
5,-1,0,-1,1,-1,-1,1,1,-1,1,1,-1,1,0,0,-1,-1,-1,0,1,1,1,1,1,1,1,-1,1,-1,-1,1
6,1,0,-1,1,1,-1,-1,-1,1,1,1,1,-1,-1,0,-1,-1,-1,0,1,1,1,1,1,-1,-1,-1,1,0,-1,-1
7,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,0,1,-1
8,1,0,-1,1,1,-1,1,1,-1,1,1,-1,1,0,1,-1,1,1,0,1,1,1,1,1,-1,1,1,1,0,1,1
9,1,1,-1,1,1,-1,-1,1,-1,1,1,1,1,0,1,-1,1,1,0,1,1,1,1,1,-1,0,-1,1,0,1,-1


In [5]:
def profile(df: pd.DataFrame) -> pd.DataFrame:
    dtypes = df.dtypes.astype(str).rename("dtype")
    missing = (df.isna().sum()).rename("missing_count")
    missing_pct = (df.isna().mean() * 100).round(2).rename("missing_pct")
    nunique = df.nunique(dropna=True).rename("nunique")
    prof = pd.concat([dtypes, missing, missing_pct, nunique], axis=1)
    prof.index.name = "column"
    return prof.sort_values(["missing_pct", "nunique"], ascending=[False, True])

profiles = {name: profile(df) for name, df in dfs.items()}
for name, prof in profiles.items():
    display(prof.style.set_caption(f"{name} — dtype • missing • nunique"))


,dtype,missing_count,missing_pct,nunique
column,,,,
chDI_SPwave,int64,0,0.000000,1
chAVL_SPwave,int64,0,0.000000,1
chAVL_RRwaveExists,int64,0,0.000000,1
chAVF_RPwaveExists,int64,0,0.000000,1
chV4_RPwaveExists,int64,0,0.000000,1
chV4_DD_RPwaveExists,int64,0,0.000000,1
chV5_SPwave,int64,0,0.000000,1
chV5_RRwaveExists,int64,0,0.000000,1
chV5_RPwaveExists,int64,0,0.000000,1


,dtype,missing_count,missing_pct,nunique
column,,,,
Class,int64,0,0.000000,2
Mitoses,int64,0,0.000000,9
Clump Thickness,int64,0,0.000000,10
Uniformity of Cell Size,int64,0,0.000000,10
Uniformity of Cell Shape,int64,0,0.000000,10
Marginal Adhesion,int64,0,0.000000,10
Single Epithelial Cell Size,int64,0,0.000000,10
Bare Nuclei,int64,0,0.000000,10
Bland Chromatin,int64,0,0.000000,10


,dtype,missing_count,missing_pct,nunique
column,,,,
having_IP_Address,int64,0,0.000000,2
Shortining_Service,int64,0,0.000000,2
having_At_Symbol,int64,0,0.000000,2
double_slash_redirecting,int64,0,0.000000,2
Prefix_Suffix,int64,0,0.000000,2
Domain_registeration_length,int64,0,0.000000,2
Favicon,int64,0,0.000000,2
port,int64,0,0.000000,2
HTTPS_token,int64,0,0.000000,2
